In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
indep = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dep = dataset['Purchased']
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],'splitter':['best', 'random'],
              'max_features':['sqrt', 'log2']}

grid = GridSearchCV (DecisionTreeClassifier(), param_grid, refit = True, verbose = 3, n_jobs = -1, scoring = 'f1_weighted')
grid.fit(X_train,Y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [13]:
#print best parameter after tuning
#print(grid.best_params_)
re = grid.cv_results_
grid_predictions = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_predictions)

In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test,grid_predictions)

In [15]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'random'}: 0.9111247448089108


In [16]:
print(cm)

[[77  8]
 [ 4 45]]


In [17]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93        85
           1       0.85      0.92      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9121248499399759

In [19]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005799,0.000981,0.009599,0.001020,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.888107,0.795256,0.851527,0.865054,0.845242,0.032026,6
1,0.004056,0.000509,0.009599,0.000800,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.885356,0.849057,0.795256,0.886792,0.924528,0.868198,0.043589,1
2,0.004000,0.000001,0.009000,0.000001,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.826263,0.889022,0.851527,0.832483,0.865054,0.852870,0.022719,4
3,0.003946,0.000721,0.009600,0.001855,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.787654,0.888107,0.832483,0.850543,0.805463,0.832850,0.035098,11
4,0.005002,0.001096,0.009243,0.001123,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.831253,0.796284,0.886792,0.862442,0.844782,0.030389,7
5,0.005199,0.000748,0.010199,0.002638,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.808927,0.886792,0.851527,0.811321,0.903610,0.852436,0.038428,5
6,0.004659,0.000426,0.009199,0.002481,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.804764,0.868752,0.814409,0.851527,0.868632,0.841617,0.027066,8
7,0.005400,0.002799,0.008399,0.000799,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.776290,0.866968,0.811321,0.850543,0.883278,0.837680,0.038900,10
8,0.004800,0.000400,0.009400,0.001020,log_loss,sqrt,best,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.847141,0.850809,0.833323,0.886792,0.847020,0.853017,0.017911,3
9,0.004000,0.000633,0.010399,0.002244,log_loss,sqrt,random,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.847141,0.849057,0.796284,0.833323,0.867097,0.838580,0.023719,9


In [21]:
age_input = float(input("Age:"))
salary_input = float(input("Salary:"))
sex_male_input = int(input("sex_male 0 or 1:"))
Future_Prediction = grid.predict([[age_input, salary_input, sex_male_input]])
Future_Prediction

Age: 32
Salary: 121323
sex_male 0 or 1: 1


array([1], dtype=int64)